In [1]:
import os
import numpy as np
import mxnet as mx
from model.simple_stack import SimpleStack
from utils import check_dir
from memory import Memory
from environments.SimpleEnv import SimpleEnv
from utils import create_input, translate_state
from evaluation import evaluate
from mxnet import gluon, nd, autograd

In [2]:
# training cases
order = "TEST"
# batch size
batch_size = 512
# agent view
agent_view = 5
map_size = 10
# action max
action_max = 3
# learning rate
model_save = "./model_save/"
lr = 0.005
num_episode = 1000000
# start play
replay_start = 10000
# update step
update_step = 1000
# gamma in q-loss calculation
gamma = 0.99
# memory pool size
memory_length = 100000
# file to save train log
summary = "./{}_Reward.csv".format(order)
eval_statistics = "./{}_CSV.csv".format(order)
# the number of step it take to linearly anneal the epsilon to it min value
annealing_end = 200000
# min level of stochastically of policy (epsilon)-greedy
epsilon_min = 0.2
# temporary files
temporary_model = "./{}/{}.params".format(model_save, order)
temporary_pool = "./{}/{}.pool".format(model_save, order)

In [3]:
if os.path.exists(summary):
    os.remove(summary)
ctx = mx.gpu()
for i in ["model_save", "data_save"]:
    check_dir(i)

In [4]:
# build models
online_model = SimpleStack(agent_view, map_size)
offline_model = SimpleStack(agent_view, map_size)
online_model.collect_params().initialize(mx.init.Normal(0.02), ctx=ctx)
offline_model.collect_params().initialize(mx.init.Normal(0.02), ctx=ctx)
offline_model.collect_params().zero_grad()

[00:43:54] ../src/base.cc:80: cuDNN lib mismatch: linked-against version 8101 != compiled-against version 8100.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [5]:
# create env
env = SimpleEnv(display=False, agent_view=agent_view)
env.reset_env()
memory_pool = Memory(memory_length, ctx=ctx)
annealing = 0
total_reward = np.zeros(num_episode)
eval_result = []
loss_func = gluon.loss.L2Loss()
trainer = gluon.Trainer(offline_model.collect_params(), 'adam', {'learning_rate': lr})

In [6]:
_epoch = 0

In [10]:
data = create_input([translate_state(env.map.state())], ctx)

In [11]:
data

[
 [[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 
   [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 
   [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 
   [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 
   [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
    [1. 0. 0. 0. 0. 

In [7]:
_print = True
for epoch in range(num_episode):
    env.reset_env()
    finish = 0
    cum_clipped_dr = 0
    if epoch == 51:
        print("Model Structure: ")
        print(offline_model)
    if sum(env.step_count) > replay_start and _print:
        print('annealing and learning are started')
        _print = False
    while not finish:
        if sum(env.step_count) > replay_start:
            annealing += 1
        eps = np.maximum(1 - sum(env.step_count) / annealing_end, epsilon_min)
        if np.random.random() < eps:
            by = "Random"
            action = np.random.randint(0, action_max)
        else:
            by = "Model"
            data = create_input([translate_state(env.map.state())], ctx)
            action = offline_model(data)
            action = int(nd.argmax(action, axis=1).asnumpy()[0])
        old, new, reward_get, finish = env.step(action)
        memory_pool.add(old, new, action, reward_get, finish)
        if finish and epoch > 50:
            cum_clipped_dr += env.detect_rate[-1]
            dr_50 = float(np.mean(env.detect_rate[-50:]))
            dr_all = float(np.mean(env.detect_rate))
            if epoch % 50 == 0:
                text = "DR: %f(50), %f(all), eps: %f" % (dr_50, dr_all, eps)
                print(text)
                with open(summary, "a") as f:
                    f.writelines(text + "\n")
            if epoch % 100 == 0 and annealing > replay_start:
                eval_result.extend(evaluate(ctx, offline_model, agent_view, 5))
        # save model and replace online model each update_step
        if annealing > replay_start and annealing % update_step == 0:
            offline_model.save_parameters(temporary_model)
            online_model.load_parameters(temporary_model, ctx)
    #  train every 2 epoch
    if annealing > replay_start and epoch % 2 == 0:
        # Sample random mini batch of transitions
        if len(memory_pool.memory) > batch_size:
            bz = batch_size
        else:
            bz = len(memory_pool.memory)
        for_train = memory_pool.next_batch(bz)
        with autograd.record(train_mode=True):
            q_sp = nd.max(online_model(for_train["state_next"]), axis=1)
            q_sp = q_sp * (nd.ones(bz, ctx=ctx) - for_train["finish"])
            q_s_array = offline_model(for_train["state"])
            q_s = nd.pick(q_s_array, for_train["action"], 1)
            loss = nd.mean(loss_func(q_s, (for_train["reward"] + gamma * q_sp)))
        loss.backward()
        trainer.step(bz)
    total_reward[int(epoch) - 1] = cum_clipped_dr

Model Structure: 
SimpleStack(
  (view_decode): Sequential(
    (0): Dense(300 -> 1024, linear)
    (1): Dense(1024 -> 1024, linear)
    (2): Dense(1024 -> 128, linear)
    (3): Dense(128 -> 128, linear)
  )
  (map_decode): Sequential(
    (0): Dense(4800 -> 1024, linear)
    (1): Dense(1024 -> 1024, linear)
    (2): Dense(1024 -> 128, linear)
    (3): Dense(128 -> 128, linear)
  )
  (decision_making): Sequential(
    (0): Dense(260 -> 2048, linear)
    (1): Dense(2048 -> 2048, linear)
    (2): Dense(2048 -> 512, linear)
    (3): Dense(512 -> 128, linear)
    (4): Dense(128 -> 64, linear)
    (5): Dense(64 -> 3, Activation(relu))
  )
)
DR: 0.053515(50), 0.060581(all), eps: 0.950000
annealing and learning are started
DR: 0.048430(50), 0.056558(all), eps: 0.925000
DR: 0.043362(50), 0.053275(all), eps: 0.900000
DR: 0.051937(50), 0.053009(all), eps: 0.875000
DR: 0.096625(50), 0.060254(all), eps: 0.850000
DR: 0.079399(50), 0.062981(all), eps: 0.825000
DR: 0.108484(50), 0.068655(all), eps: 0

KeyboardInterrupt: 